In [11]:
import networkx as nx
import ast
import numpy as np
from scipy import sparse

In [12]:
g = nx.read_gml('testCinfExport.gml')

In [13]:
nx.to_dict_of_dicts(g)

{'[1, 2, 3]': {'[2, 1, 3]': {'weight': '[[1, 0, 0], [0, 0, 0]]'},
  '[1, 3, 2]': {'weight': '[[0, 1, 0], [0, 0, 0]]'},
  '[3, 2, 1]': {'weight': '[[0, 0, 1], [0, 0, 0]]'}},
 '[2, 1, 3]': {'[1, 2, 3]': {'weight': '[[0, 0, 0], [0, 0, 1]]'},
  '[2, 3, 1]': {'weight': '[[0, 0, 1], [0, 0, 0]]'},
  '[3, 1, 2]': {'weight': '[[0, 1, 0], [1, 0, 0]]'}},
 '[2, 3, 1]': {'[3, 2, 1]': {'weight': '[[0, 0, 0], [1, 0, 0]]'},
  '[2, 1, 3]': {'weight': '[[0, 0, 0], [0, 1, 0]]'},
  '[1, 3, 2]': {'weight': '[[0, 0, 0], [0, 0, 1]]'}},
 '[3, 2, 1]': {'[2, 3, 1]': {'weight': '[[0, 1, 0], [0, 0, 0]]'},
  '[3, 1, 2]': {'weight': '[[1, 0, 0], [0, 0, 1]]'},
  '[1, 2, 3]': {'weight': '[[0, 0, 0], [0, 1, 0]]'}},
 '[3, 1, 2]': {'[1, 3, 2]': {'weight': '[[0, 0, 0], [0, 0, 0]]'},
  '[3, 2, 1]': {'weight': '[[0, 0, 0], [0, 0, 0]]'},
  '[2, 1, 3]': {'weight': '[[0, 0, 0], [0, 0, 0]]'}},
 '[1, 3, 2]': {'[3, 1, 2]': {'weight': '[[0, 0, 1], [0, 1, 0]]'},
  '[1, 2, 3]': {'weight': '[[0, 0, 0], [1, 0, 0]]'},
  '[2, 3, 1]': {

In [27]:
def convert_gml(fileName): #GridToolsTBD exports to a text file - this grabs it and converts the edges back to lists
                           #super inneficient - should convert to pickle file system or something
    g = nx.read_gml(fileName)
    for edge in g.edges:
        g[edge[0]][edge[1]]['weight'] = ast.literal_eval(g[edge[0]][edge[1]]['weight'])
        g[edge[0]][edge[1]]['weight'] = g[edge[0]][edge[1]]['weight'][0]+g[edge[0]][edge[1]]['weight'][1] #end result is edge weights as list of
                                                                                                          #multiplicities [U1,U2...,Un,V1,...,Vn]
    return g

def construct_cinf(g, size = -1): #Construct CFKinf complex from graph data - essentially just changing weights to polynomials
                                  #Only works for grid diagrams *not* Latin Squares
    if size == -1:
        size = len(g.get_edge_data(list(g.edges())[0][0],list(g.edges())[0][1])['weight'])  #kind of a mess - just turning the edges
    n = size/2                                                                              #into a list and checking the length of
                                                                                            #the weight of the first edge
    F,Vars = cinf_coeff(n)
    sageG = DiGraph()
    for edge in g.edges:
        
        start = edge[0]
        end = edge[1]
        poly = F(1)
        i = 0
        for entry in g[edge[0]][edge[1]]['weight']:
            
            poly = poly*(Vars[i])**entry
            i = i + 1
            
#        g[start][end]['weight'] = poly
        sageG.add_edge(start,end,poly)
    
    return sageG
        
def cinf_coeff(size):
    n = size
    preF = LaurentPolynomialRing(GF(2), 'U', n) #F[Ui+-] which we'll then pair up with the Vi
    preF.inject_variables()                     #Telling Sage we have Ui's as variables
    Vars = preF.gens()                          #storing the variables in a list - not currently implemented anywhere
    F = LaurentPolynomialRing(preF, 'V', n)     #Takes our preF (F[Ui+-]) and adjoins Vi
    F.inject_variables()                        #F only thinks it has Vi as variables, we tell Sage about it
    Vars = Vars + F.gens()
    return F,Vars

def find_one(targetlist): #Searches a list for first 1 - will be used for reduction
#     print("searching for 1 in" + str(targetlist))
    if 1 in list(targetlist):
#         print("found 1 in the list at" + str(list(targetlist).index(1)))
        return list(targetlist).index(1)
    return -1

def find_col_with_one(matrix, startc=0):
    endc = len(matrix[0])
    print(str(endc))
    for n in range(startc, endc):
        search_result = find_one(matrix[:][n])
        if search_result != -1:
            return (search_result, n)
    return (-1, -1)


def reduction_remap(matrix, row, col):
    n = len(matrix[0])
    range1 = range_skip_entry(n, row)
#     print("searching through " + str(range1))
    for count, target_row in enumerate(range1):
        entry = matrix[target_row][col]
        if entry != 0:
#             print("doing row addition with row" + str(row))
#             print("and count" + str(count))
#             print("and entry (multiple)" + str(entry))
            my_row_add(matrix, row, target_row, entry)
    return matrix

def row_col_del(matrix, loc):
    newrange = list(range(len(matrix[0])))
    del newrange[loc]
    return matrix[newrange,newrange]

def range_skip_entry(n, skip):
    u = []
    for i in range(0, skip):
        u.append(i)
    for j in range(skip+1, n):
        u.append(j)
    return u

def reduction(matrix):
    col, row = find_col_with_one(matrix)
#     print("reducing around entry " +str(row) + "," +str(col))
    if col == -1:
        print("completed reduction")
        print(matrix)
        print (type(matrix))
        return matrix
    print("reduction in progress")
    remapped_matrix = reduction_remap(matrix, row, col)
    if row < col:
        remapped_matrix = row_col_del(remapped_matrix, col)
        remapped_matrix = row_col_del(remapped_matrix, row)
    else:
        remapped_matrix = row_col_del(remapped_matrix, row)
        remapped_matrix = row_col_del(remapped_matrix, col)
    return reduction(remapped_matrix)
    
def my_row_add(matrix, row, targetrow, multiple):
    n = len(matrix[0])
    for i in range(n):
        current_src = matrix[row][i]
        if current_src != 0:
#             print("adding copies times " + str(current_src))
#             print("multiple" + str(multiple))
#             print("target entry " + str(matrix[targetrow][i]))
            matrix[targetrow,i] = matrix[targetrow][i] + multiple*current_src
#         print(matrix)
    return matrix
        

iterate through dictionary keys(dict)
    tracker = -1
    for target in keydic
        if target weight == 1
            graph reduction alg
            tracker = 1
    if tracker == 1
        return rerun
    else
        return
        
graph reduction(dict, key, target)
    for x in predecessors(target)
        if x == key: continue
        for y in successors(key)
            if y == target: continue
            x_weight = thegraph[x][targ]['weight']
            y_weight = thegraph[key][y]['weight']
            W = x_weight x y_weight
            add edge to graph from x to y weight = W
    delete key
    delete target
    return graph

In [57]:
def graph_red_search(given_graph): 
#searches through a cfk inf complex for reducible edges and calling
#the reduction function to eliminate the pair according to the reduction algorithm
#     dict_graph = nx.to_dict_of_dicts(given_graph)
    for key in given_graph:
        
        for target in given_graph[key]:
            
            if given_graph[key][target]['weight'] == 1:
                given_graph = graph_reduction(given_graph, key, target)
                return graph_red_search(given_graph)
            
    return given_graph

def graph_reduction(given_graph, key, target):
#Deletes edge specified from graph_red_search and adds in edges according to the
#reduction algorithm
    for x in given_graph.predecessors(target):
        
        if x == key: continue
        for y in given_graph.successors(key):
        
            if y == target: continue
            x_weight = given_graph[x][target]['weight']
            y_weight = given_graph[key][y]['weight']
            red_weight = x_weight * y_weight
            given_graph.add_edge(x,y,weight=red_weight)

    given_graph.remove_node(x)
    given_graph.remove_node(y)
    return given_graph



In [58]:
thering = cinf_coeff(3)

Defining U0, U1, U2
Defining V0, V1, V2


In [59]:
g = convert_gml('testCinfExport.gml')
sageG = construct_cinf(g)
dictg = nx.to_dict_of_dicts(g)
# newg = DiGraph()
#graphs.graph_input.from_dict_of_dicts(dictg, weighted = True)
# newg
myf = sageG.weighted_adjacency_matrix()

Defining U0, U1, U2
Defining V0, V1, V2


In [36]:
nxg = sageG.networkx_graph()

In [60]:
redg = graph_red_search(nxg)

In [61]:
redg.edges()

OutEdgeView([('[1, 2, 3]', '[2, 1, 3]'), ('[1, 2, 3]', '[1, 3, 2]'), ('[2, 1, 3]', '[1, 2, 3]'), ('[2, 1, 3]', '[2, 3, 1]'), ('[1, 3, 2]', '[1, 2, 3]'), ('[1, 3, 2]', '[2, 3, 1]'), ('[2, 3, 1]', '[2, 1, 3]'), ('[2, 3, 1]', '[1, 3, 2]')])

In [62]:
redg.nodes()

NodeView(('[1, 2, 3]', '[2, 1, 3]', '[1, 3, 2]', '[2, 3, 1]'))

In [64]:
redg.get_edge_data('[1, 2, 3]', '[2, 1, 3]')

{'weight': U0}

In [55]:
g

In [49]:
print(sparse.coo_matrix(myf))

/opt/sagemath-9.3/local/lib/python3.7/site-packages/scipy/sparse/sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


ValueError: cannot create object arrays from iterator

In [42]:
testy = lil_array(myf)

NameError: name 'lil_array' is not defined

In [30]:
testy.todok

TypeError: 'coo_matrix' object is not subscriptable

In [14]:
type(myf)

<class 'sage.matrix.matrix_generic_sparse.Matrix_generic_sparse'>

In [15]:
trefred = reduction(myf)

6
reduction in progress
4
completed reduction
[      0 U0 + U1       0 U1 + U2]
[     V2       0      U2       0]
[      0 V1 + V2       0 V0 + V2]
[     V1       0      U1       0]
<class 'sage.matrix.matrix_generic_sparse.Matrix_generic_sparse'>


In [20]:
trefred

[      0 U0 + U1       0 U1 + U2]
[     V2       0      U2       0]
[      0 V1 + V2       0 V0 + V2]
[     V1       0      U1       0]

In [96]:
idk = np.array(trefred)

In [99]:
idk

array([[0, V2, V1, ..., 0, 0, 0],
       [U4, 0, 0, ..., 0, 0, 0],
       [U3, 0, 0, ..., 0, 0, V3],
       ...,
       [0, 0, 0, ..., 0, 0, U2],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, U0 + U3, ..., V2, V0, 0]], dtype=object)

In [98]:
nx.Graph(idk)

NetworkXError: Input is not a correct numpy matrix or array.

In [80]:
myf

120 x 120 sparse matrix over Multivariate Laurent Polynomial Ring in V0, V1, V2, V3, V4 over Multivariate Laurent Polynomial Ring in U0, U1, U2, U3, U4 over Finite Field of size 2 (use the '.str()' method to see the entries)

In [61]:
testmat = matrix([[1,0,U1],[V0,1,V2],[U0,U1,V2]])
testmat

[ 1  0 U1]
[V0  1 V2]
[U0 U1 V2]

In [45]:
my_row_add(testmat, 0, 1, V0)

adding copies times 1
multipleV0
target entry V0
[ 1  0 U1]
[ 0  1 V2]
[U0 U1 V2]
[ 1  0 U1]
[ 0  1 V2]
[U0 U1 V2]
adding copies times U1
multipleV0
target entry V2
[         1          0         U1]
[         0          1 U1*V0 + V2]
[        U0         U1         V2]


[         1          0         U1]
[         0          1 U1*V0 + V2]
[        U0         U1         V2]

In [62]:
reduction_remap(testmat,0,0)

searching through [1, 2]
doing row addition with row0
and count0
and entry (multiple)V0
adding copies times 1
multipleV0
target entry V0
[ 1  0 U1]
[ 0  1 V2]
[U0 U1 V2]
[ 1  0 U1]
[ 0  1 V2]
[U0 U1 V2]
adding copies times U1
multipleV0
target entry V2
[         1          0         U1]
[         0          1 U1*V0 + V2]
[        U0         U1         V2]
doing row addition with row0
and count1
and entry (multiple)U0
adding copies times 1
multipleU0
target entry U0
[         1          0         U1]
[         0          1 U1*V0 + V2]
[         0         U1         V2]
[         1          0         U1]
[         0          1 U1*V0 + V2]
[         0         U1         V2]
adding copies times U1
multipleU0
target entry V2
[         1          0         U1]
[         0          1 U1*V0 + V2]
[         0         U1 V2 + U0*U1]


[         1          0         U1]
[         0          1 U1*V0 + V2]
[         0         U1 V2 + U0*U1]

In [64]:
test = reduction(matrix([[0,1,U1],[V0,0,V2],[U0,U1,V2]]))

3
searching for 1 in(0, 1, U1)
found 1 in the list at1
reducing around entry 0,1
reduction in progress
searching through [1, 2]
doing row addition with row0
and count1
and entry (multiple)U1
[ 0  1 U1]
[V0  0 V2]
[U0 U1 V2]
adding copies times 1
multipleU1
target entry U1
[ 0  1 U1]
[V0  0 V2]
[U0  0 V2]
adding copies times U1
multipleU1
target entry V2
[        0         1        U1]
[       V0         0        V2]
[       U0         0 V2 + U1^2]
1
searching for 1 in(V2 + U1^2)
reducing around entry -1,-1
completed reduction
[V2 + U1^2]
<class 'sage.matrix.matrix_generic_dense.Matrix_generic_dense'>


In [107]:
myf.add_multiple_of_row(1,2,U1)

In [108]:
myf

[              0              U1              U0               0               0              U2]
[     V0 + U1*V2               0               0      U1*U2 + U0 U1^2*V0 + U2*V1               0]
[             V2               0               0              U2           U1*V0               0]
[              0              V2              V1               0               0              V0]
[              0               1               1               0               0               1]
[             V1               0               0              U1           U0*V2               0]

In [109]:
1 in GF(3)

True

In [110]:
1 in thering

False

In [111]:
myf[0,1] in thering

False

In [112]:
find_col_with_one(myf)

6
searching for 1 in(0, U1, U0, 0, 0, U2)
searching for 1 in(V0 + U1*V2, 0, 0, U1*U2 + U0, U1^2*V0 + U2*V1, 0)
searching for 1 in(V2, 0, 0, U2, U1*V0, 0)
searching for 1 in(0, V2, V1, 0, 0, V0)
searching for 1 in(0, 1, 1, 0, 0, 1)
found 1 in the list at1


(1, 4)

In [116]:
newmat = reduction(myf)

6
searching for 1 in(0, U1, U0, 0, 0, U2)
searching for 1 in(V0, 0, 0, U0, U2*V1, 0)
searching for 1 in(V2, 0, 0, U2, U1*V0, 0)
searching for 1 in(0, V2, V1, 0, 0, V0)
searching for 1 in(0, 1, 1, 0, 0, 1)
found 1 in the list at1
reducing around entry 4,1
reduction in progress
searching through [0, 1, 2, 3, 5]
doing row addition with row4
and count0
and entry (multiple)V0
[    0    U1    U0     0     0    U2]
[   V0     0     0    U0 U2*V1     0]
[   V2     0     0    U2 U1*V0     0]
[    0    V2    V1     0     0    V0]
[    0     1     1     0     0     1]
[   V1     0     0    U1 U0*V2     0]
adding copies times 1
multipleV0
target entry U1
[      0 V0 + U1      U0       0       0      U2]
[     V0       0       0      U0   U2*V1       0]
[     V2       0       0      U2   U1*V0       0]
[      0      V2      V1       0       0      V0]
[      0       1       1       0       0       1]
[     V1       0       0      U1   U0*V2       0]
adding copies times 1
multipleV0
target entry U0


In [145]:
print(newmat)

[     0 U0 + 1      0 U2 + 1]
[    V2      0     U2      0]
[     0 V1 + 1      0 V0 + 1]
[    V1      0     U1      0]


In [59]:
myf

[        0        U1        U0         0         0        U2]
[       V0         0         0        U0     U2*V1         0]
[       V2         0         0        U2     U1*V0         0]
[        0        V2        V1         0         0        V0]
[        0  U1^2 + 1 U0*U1 + 1         0         0 U1*U2 + 1]
[       V1         0         0        U1     U0*V2         0]

In [58]:
print(newmat)

None


In [32]:
range_skip_entry(5,2)

[0, 1, 3, 4]

In [33]:
g.get_edge_data(list(g.edges())[0][0],list(g.edges())[0][1])['weight']

[1, 0, 0, 0, 0, 0]

In [98]:
preF = LaurentPolynomialRing(GF(2), 4, ('w'))
preF.inject_variables()
F = LaurentPolynomialRing(preF,4,'z')
F.inject_variables()
F.gens()[0]

Defining w0, w1, w2, w3
Defining z0, z1, z2, z3


z0

In [165]:
preF = LaurentPolynomialRing(GF(2), 'U', 3)
preF.inject_variables()
F = LaurentPolynomialRing(GF(2), 'V', 3)
F.inject_variables()

Defining U0, U1, U2
Defining V0, V1, V2


In [16]:
F

Multivariate Laurent Polynomial Ring in U, V over Finite Field of size 2

In [ ]:
#OLD CODE

# def reduce_around(M, position): 
# #M isa matrix and position is a pair [a,b] where a 1 is located
# #function does Gauss-Jordan-ish elimination on that column and in a symmetric way to the entry's row
#     a,b = position
#     #First step is to use the row to cancel the other entries, then we'll do a symmetric cancellation on
#     #the columns as well
#     column = M[:][b]
#     for index, entry in enumerate(column):
#         if ((index != 0) and (index != a) and (entry != 0)):
#             M.add_multiple_of_row(index, entry, a)
#     for index, entry in enumerate(column):
#         if ((index != 0) and (index != a) and (entry != 0)):
#             M.add_multiple_of_col(index, entry, b)
#     #Now we'll repeat the process but with the row entries
#     row = M[a][:]
#     for index, entry in enumerate(row):
#         if ((index != 0) and (index != b) and (entry != 0)):
#             M.add_multiple_of_col(index, entry, b)
#     for index, entry in enumerate(column):
#         if ((index != 0) and (index != b) and (entry != 0)):
#             M.add_multiple_of_row(index, entry, a)
#     return

# def hom_reduction(adj_mat):
#     M = adj_mat
#     row_position = 0
#     for row in M:
#         check, column_position = row_count(row)
#         if row_count(check[0]) > 1:
#             reduce_around(M, row_position, column_position)
#         row_position = position + 1